In [2]:
# pip install pandas pymongo python-dateutil
import json
from datetime import datetime
from dateutil.tz import tzutc
from pymongo import MongoClient, UpdateOne
import numpy as np
from pathlib import Path
import pandas as pd

# ---------- helpers ----------
def to_dt(s):
    """Parse ISO or {dueDate, dueTime} dicts into a Python datetime (UTC)."""
    if s is None:
        return None
    if isinstance(s, str):
        try:
            # handle '2025-08-30T19:27:44.005Z'
            return datetime.fromisoformat(s.replace("Z", "+00:00")).astimezone(tzutc())
        except Exception:
            return None
    if isinstance(s, dict) and {"year","month","day"} <= set(s.get("dueDate", s).keys()):
        d = s.get("dueDate", s)
        t = s.get("dueTime", {})
        hh, mm = int(t.get("hours", 0)), int(t.get("minutes", 0))
        return datetime(int(d["year"]), int(d["month"]), int(d["day"]), hh, mm, tzinfo=tzutc())
    return None

def df_to_docs(df: pd.DataFrame):
    """NaN->None and convert DataFrame to list of dicts."""
    if df is None or df.empty:
        return []
    return df.replace({np.nan: None}).to_dict("records")

# ---------- load ----------

json_directory = "../classroom_data"

json_files = sorted(Path(json_directory).glob("classroom_data_*.json"),reverse=True)

with open(json_files[0],"r",encoding="UTF-8") as f:
    raw = json.load(f)
    
courses_raw = raw["courses"]

# ---------- build DataFrames with json_normalize ----------
# Courses (flat course_info)
courses_df = pd.json_normalize(courses_raw, record_path=None, meta=None)

# courses_df = pd.json_normalize(courses_df["course_info"])  # keep just course_info
courses_df.rename(columns={"course_info.id":"courseId"}, inplace=True)
courses_df["creationTime"] = courses_df["course_info.creationTime"].map(to_dt)
courses_df["updateTime"]  = courses_df["course_info.updateTime"].map(to_dt)
courses_df.drop(columns="course_info.creationTime", axis= 1 , inplace=True)
courses_df.drop(columns="course_info.updateTime", axis= 1 , inplace=True)


courses_df

    
# Students

,students,teachers,assignments,announcements,courseId,course_info.name,course_info.section,course_info.ownerId,course_info.enrollmentCode,course_info.courseState,...,course_info.courseGroupEmail,course_info.teacherFolder.id,course_info.teacherFolder.title,course_info.teacherFolder.alternateLink,course_info.guardiansEnabled,course_info.calendarId,course_info.gradebookSettings.calculationType,course_info.gradebookSettings.displaySetting,creationTime,updateTime
0,"[{'courseId': '775041576431', 'userId': '10285...","[{'courseId': '775041576431', 'userId': '10271...","[{'courseId': '775041576431', 'id': '775044007...",[],775041576431,CAI3_AIS5_S2,Data Engineering,102712182136620142659,n3oyqgx7,ACTIVE,...,CAI3_AIS5_S2_Data_Engineering_eeeb99cc@classro...,1TA3nwSqRpTQTQ-WG8k0Ms2Bu3d7P6WUvNnGYrwxdQ5O8w...,CAI3_AIS5_S2 Data Engineering,https://drive.google.com/drive/folders/1TA3nwS...,False,classroom114502311760584562642@group.calendar....,TOTAL_POINTS,HIDE_OVERALL_GRADE,2025-08-30 19:27:44.005000+00:00,2025-08-30 19:27:44.005000+00:00
1,"[{'courseId': '736229168930', 'userId': '10824...","[{'courseId': '736229168930', 'userId': '10271...","[{'courseId': '736229168930', 'id': '757760276...",[],736229168930,IBM-Data Science -GIZ2_AIS4_S2,NaN,102410293651192205486,mmrlu7b,ACTIVE,...,IBM_Data_Science_GIZ2_AIS4_S2_93a4f458@classro...,1IyezaR8zL0v7WHqwl_nppjWuWdJNfhIn5PW-bz-_p2bpi...,IBM-Data Science -GIZ2_AIS4_S2,https://drive.google.com/drive/folders/1IyezaR...,False,classroom100045738141013408938@group.calendar....,TOTAL_POINTS,HIDE_OVERALL_GRADE,2024-11-29 12:02:16.207000+00:00,2024-11-29 12:02:16.207000+00:00
2,"[{'courseId': '749164777280', 'userId': '11172...","[{'courseId': '749164777280', 'userId': '10271...","[{'courseId': '749164777280', 'id': '759168939...",[],749164777280,ISM2_DAT1_S1,Data Analyst Specialist,115575352399457665837,2khwp3j,ACTIVE,...,ISM2_DAT1_S1_Data_Analyst_Specialist_c89af261@...,1oIsbH8S7c0c27GQNpX1eomW0xE8qdRosTl4SOSXNgnqZL...,ISM2_DAT1_S1 Data Analyst Specialist,https://drive.google.com/drive/folders/1oIsbH8...,False,classroom103452772798036270238@group.calendar....,TOTAL_POINTS,HIDE_OVERALL_GRADE,2025-02-06 15:37:21.061000+00:00,2025-02-06 15:37:21.061000+00:00
3,"[{'courseId': '730824172977', 'userId': '11610...","[{'courseId': '730824172977', 'userId': '10271...","[{'courseId': '730824172977', 'id': '699054875...","[{'courseId': '730824172977', 'id': '777440384...",730824172977,Microsoft Machine Learning - BNS2_AIS2_S1,NaN,102410293651192205486,qgex77p,ACTIVE,...,Microsoft_Machine_Learning_440903a0@classroom....,135rwOGsmBb1AtDxmecZuFb3474SBUh-IyPXyebuy_2Hy1...,Microsoft Machine Learning,https://drive.google.com/drive/folders/135rwOG...,False,classroom102637974415858229413@group.calendar....,TOTAL_POINTS,SHOW_OVERALL_GRADE,2024-11-12 15:32:37.854000+00:00,2025-03-10 10:43:34.254000+00:00
4,"[{'courseId': '730741605390', 'userId': '11728...","[{'courseId': '730741605390', 'userId': '10271...","[{'courseId': '730741605390', 'id': '764775268...","[{'courseId': '730741605390', 'id': '775654341...",730741605390,IBM Data Science-CAI2_AIS4_S15,NaN,102410293651192205486,xojo34w,ACTIVE,...,IBM_Data_Science_7ec2c43a@classroom.google.com,1j3kR54uSXAnnTaa8yYXlePEoYHmq7MrpvSlZ-UtvOUnmr...,IBM Data Science,https://drive.google.com/drive/folders/1j3kR54...,False,classroom114511228298132539175@group.calendar....,TOTAL_POINTS,SHOW_OVERALL_GRADE,2024-11-12 12:33:12.823000+00:00,2024-12-01 12:34:36.219000+00:00
5,"[{'courseId': '749614754622', 'userId': '10628...","[{'courseId': '749614754622', 'userId': '10271...",[],[],749614754622,BNI_DATA1_S1,Power BI Engineer,115575352399457665837,c6ctnmd,ACTIVE,...,BNI_DATA1_S1_Power_BI_Engineer_2bc7a28a@classr...,1zBvwkgnfWsoQcCj32X55_LpD5OIr3mVh6GWW8rU34BJcD...,BNI_DATA1_S1 Power BI Engineer,https://drive.google.com/drive/folders/1zBvwkg...,False,classroom102760313450491861621@group.calendar....,TOTAL_POINTS,HIDE_OVERALL_GRADE,2025-02-10 09:42:48.244000+00:00,2025-02-10 09:42:48.24

In [20]:
# Students
students_df = pd.json_normalize(
    courses_raw,
    record_path=["students"],
    meta=[["course_info","id"]],
    sep="."
)
# students_df.rename(columns={"course_info.id":"courseId"}, inplace=True)
students_df.drop(columns="profile.id",axis=1,inplace=True)
students_df.drop(columns="profile.permissions",axis=1,inplace=True)
students_df.drop(columns="profile.name.givenName",axis=1,inplace=True)
students_df.drop(columns="profile.name.familyName",axis=1,inplace=True)
students_df.drop(columns="course_info.id",axis=1,inplace=True)

# Teachers
teachers_df = pd.json_normalize(
    courses_raw,
    record_path=["teachers"],
    meta=[["course_info","id"]],
    sep="."
)
# teachers_df.rename(columns={"course_info.id":"courseId"}, inplace=True)
teachers_df.drop(columns="profile.id",axis=1,inplace=True)
teachers_df.drop(columns="course_info.id",axis=1,inplace=True)
teachers_df.drop(columns="profile.name.givenName",axis=1,inplace=True)
teachers_df.drop(columns="profile.permissions",axis=1,inplace=True)
teachers_df.drop(columns="profile.name.familyName",axis=1,inplace=True)

teachers_df

,courseId,userId,profile.name.fullName,profile.emailAddress,profile.photoUrl
0,775041576431,102712182136620142659,EduLink,edu.link.org@gmail.com,//lh3.googleusercontent.com/a/ACg8ocK0JP0HiKIU...
1,736229168930,102712182136620142659,EduLink,edu.link.org@gmail.com,//lh3.googleusercontent.com/a/ACg8ocK0JP0HiKIU...
2,736229168930,102410293651192205486,Ahmed Hekal,AhmedAdelHekal@gmail.com,//lh3.googleusercontent.com/a/ACg8ocLiPW3xkt1W...
3,749164777280,102712182136620142659,EduLink,edu.link.org@gmail.com,//lh3.googleusercontent.com/a/ACg8ocK0JP0HiKIU...
4,749164777280,102410293651192205486,Ahmed Hekal,AhmedAdelHekal@gmail.com,//lh3.googleusercontent.com/a/ACg8ocLiPW3xkt1W...
5,749164777280,115575352399457665837,HebaNashy,hebanashy035@gmail.com,//lh3.googleusercontent.com/a/ACg8ocIhg3mKAsi9...
6,730824172977,102712182136620142659,EduLink,edu.link.org@gmail.com,//lh3.googleusercontent.com/a/ACg8ocK0JP0HiKIU...
7,730824172977,105278864067840973755,ِAbdulla Wagih,abdullawagih1@gmail.com,//lh3.googleusercontent.com/a-/ALV-UjUZWyiQ56n...
8,730824172977,102410293651192205486,Ahmed Hekal,AhmedAdelHekal@gmail.com,//lh3.googleusercontent.com/a/ACg8ocLiPW3xkt1W...
9,730741605390,102712182136620142659,EduLink,edu.link.org@gmail.com,//lh3.googleusercontent.com/a/ACg8ocK0JP0HiKIU...


In [ ]:
assignments_df = pd.json_normalize(
    courses_raw,
    record_path=["assignments"],
    meta=[["course_info","id"]],
    sep="."
)
assignments_df.rename(columns={"id":"assignmentId"}, inplace=True)
assignments_df.drop(axis = 1 , inplace= True, columns=["courseinfo.id",""])
assignments_df

,courseId,assignmentId,title,description,materials,state,alternateLink,creationTime,updateTime,maxPoints,...,dueDate.year,dueDate.month,dueDate.day,dueTime.hours,dueTime.minutes,assignment.studentWorkFolder.id,assignment.studentWorkFolder.title,assignment.studentWorkFolder.alternateLink,topicId,courseId
0,775041576431,775044007840,Sql Assignment,Build this schema using SQL Code,[{'driveFile': {'driveFile': {'id': '1goOQQ55R...,PUBLISHED,https://classroom.google.com/c/Nzc1MDQxNTc2NDM...,2025-08-30T19:31:04.879Z,2025-08-30T19:32:40.444Z,10.0,...,2025.0,9.0,7.0,20.0,59.0,1aKHRt3zDA0TqeLJOPR9tAEKKwiRjpUb6rZODMAiHS3qgp...,Sql Assignment,https://drive.google.com/drive/folders/1aKHRt3...,NaN,775041576431
1,736229168930,757760276238,Dash Assignment,NaN,[{'driveFile': {'driveFile': {'id': '1YITDWaWv...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2025-03-07T19:40:29.605Z,2025-03-07T19:41:12.858Z,10.0,...,2025.0,3.0,14.0,21.0,59.0,12kXIOCd5GTrGoGDKfercP1HcZ4MAAbmH53QVcaAD0RrJR...,Dash Assignment,https://drive.google.com/drive/folders/12kXIOC...,757755516475,736229168930
2,736229168930,772477498441,unsupervised assignment,As mentioned in session please upload your not...,[{'driveFile': {'driveFile': {'id': '1yF7A7uc8...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2025-04-17T12:27:17.494Z,2025-04-17T12:27:16.994Z,100.0,...,NaN,NaN,NaN,NaN,NaN,1ReCb5BP0G6P2JcAEt_wQogC5pKvc4FjGs3T8lHaZzvO1J...,unsupervised assignment,https://drive.google.com/drive/folders/1ReCb5B...,NaN,736229168930
3,736229168930,772173994093,Unsupervised Machine learning,NaN,[{'driveFile': {'driveFile': {'id': '1jPausLlv...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2025-04-15T19:53:34.398Z,2025-04-15T19:53:38.485Z,100.0,...,2025.0,4.0,22.0,21.0,59.0,1JGqphWKDf3_k3JMcKTd-DF4yt0T7eISOPD6oZKs-bFGE_...,Unsupervised Machine learning,https://drive.google.com/drive/folders/1JGqphW...,761512351540,736229168930
4,736229168930,762697112016,Logistic Regression assignment,NaN,[{'driveFile': {'driveFile': {'id': '1G6gsTKfX...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2025-03-26T21:57:13.837Z,2025-03-26T22:00:49.281Z,100.0,...,2025.0,4.0,3.0,21.0,59.0,1Hl6caqqvmoVHkG7UK9OlE5y05_oHs7YFdyUjefqaeFqEO...,Logistic Regression assignment,https://drive.google.com/drive/folders/1Hl6caq...,761512351540,736229168930
5,736229168930,758959671661,Dash Assignment 2,Dataset link is attached,[{'link': {'url': 'https://www.kaggle.com/data...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2025-03-13T17:17:23.241Z,2025-03-13T17:18:08.879Z,10.0,...,NaN,NaN,NaN,NaN,NaN,1CX7B3upDsFYn2GdFSZF8B4G3c4_Xz09ohyepyrOls3LlV...,Dash Assignment 2,https://drive.google.com/drive/folders/1CX7B3u...,NaN,736229168930
6,736229168930,688269676787,pandas assignment,NaN,[{'driveFile': {'driveFile': {'id': '1KbPLZlBl...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2025-01-30T07:45:01.677Z,2025-01-30T08:19:45.183Z,10.0,...,2025.0,2.0,6.0,21.0,59.0,1Ymr2Ek9LZoPAc0HY9f-2vDnFOlL9YtKs33gUniroQlvBp...,pandas assignment,https://drive.google.com/drive/folders/1Ymr2Ek...,747804272891,736229168930
7,736229168930,748897513414,upload your work,NaN,[{'driveFile': {'driveFile': {'id': '11TXOdNfs...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2025-02-05T13:24:52.025Z,2025-02-05T13:25:22.689Z,100.0,...,NaN,NaN,NaN,NaN,NaN,1NTs41Eerx9rytaP0uU0FgwV6m6g19ypDSLK9Is_BtPc_d...,upload your work,https://drive.google.com/drive/folders/1NTs41E...,747804272891,736229168930
8,736229168930,741263859353,database quiz,NaN,[{'form': {'formUrl': 'https://docs.google.com...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2024-12-13T11:50:50.325Z,2024-12-14T07:27:30.136Z,4.0,...,NaN,NaN,NaN,NaN,NaN,1eshrGUmVwy7NrH06LJDitPEFnrh62Mdb_fs7Q7w3DjYRS...,NaN,NaN,736445337725,736229168930
9,736229168930,736445706360,Database Assignment,NaN,[{'driveFile': {'driveFile': {'id': '1TM7Ayeeh...,PUBLISHED,https://classroom.google.com/c/NzM2MjI5MTY4OTM...,2